# 基于 ResNet 的多模态医学图像分割入门教程

> 目标：给完全新手的**可跑通**教程。我们将从零开始：准备数据 → 定义数据集 → 构建模型 → 训练 → 评估 → 推理。

本 Notebook 面向**多模态**医学数据：
- **图像**（如 CT/MRI/超声）
- **结构化元数据**（如年龄、性别、模态类型、病灶大小）

我们先用“可跑通的小样例数据”，再说明如何替换成你自己的真实数据。


## 0. 环境准备（只需运行一次）

如果你在新的环境里，请先安装依赖：

```bash
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
pip install numpy pandas pillow matplotlib tqdm
```

> 如果你有 GPU，请把 `--index-url` 换成对应 CUDA 版本。


## 1. 项目与数据结构（非常重要）

我们建议你把数据按如下结构摆放：

```
examples/transformer_tutorial2/
  resnet_multimodal_seg.ipynb
  data/
    resnet_tutorial/
      images/      # 原始图像
      masks/       # 对应的分割mask（0/1 或 0~N 类）
      metadata.csv # 结构化元数据（每行对应一张图像）
```

`metadata.csv` 至少包含：
- `image_id`：文件名（不带扩展名）
- `age`、`sex`、`modality` 等你想用的字段

下面先生成一个**可运行的演示数据集**。


In [ ]:
from pathlib import Path
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw

ROOT = Path('examples/transformer_tutorial2/data/resnet_tutorial')
IMG_DIR = ROOT / 'images'
MSK_DIR = ROOT / 'masks'
ROOT.mkdir(parents=True, exist_ok=True)
IMG_DIR.mkdir(parents=True, exist_ok=True)
MSK_DIR.mkdir(parents=True, exist_ok=True)

# 生成一个小的可跑通数据集（20 张 128x128 图像）
rows = []
for i in range(20):
    img = Image.new('L', (128, 128), color=0)
    mask = Image.new('L', (128, 128), color=0)

    draw_img = ImageDraw.Draw(img)
    draw_msk = ImageDraw.Draw(mask)

    # 随机圆形“病灶”
    cx, cy = np.random.randint(32, 96, size=2)
    r = np.random.randint(10, 25)
    bbox = (cx - r, cy - r, cx + r, cy + r)
    draw_img.ellipse(bbox, fill=200)
    draw_msk.ellipse(bbox, fill=255)

    image_id = f'sample_{i:03d}'
    img.save(IMG_DIR / f'{image_id}.png')
    mask.save(MSK_DIR / f'{image_id}.png')

    rows.append({
        'image_id': image_id,
        'age': int(np.random.randint(18, 80)),
        'sex': int(np.random.randint(0, 2)),  # 0/1
        'modality': int(np.random.randint(0, 3)),  # 0:CT,1:MR,2:US
    })

meta = pd.DataFrame(rows)
meta.to_csv(ROOT / 'metadata.csv', index=False)
meta.head()


## 2. 定义数据集（图像 + mask + 元数据）

我们将图像和 mask 读入，并把元数据转成数值向量。**这一步是多模态的关键。**


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class MultimodalSegDataset(Dataset):
    def __init__(self, root_dir, img_size=128):
        self.root_dir = Path(root_dir)
        self.img_dir = self.root_dir / 'images'
        self.msk_dir = self.root_dir / 'masks'
        self.meta = pd.read_csv(self.root_dir / 'metadata.csv')

        self.transform = transforms.Compose([
            transforms.Resize((img_size, img_size)),
            transforms.ToTensor(),  # [0,1]
        ])

    def __len__(self):
        return len(self.meta)

    def __getitem__(self, idx):
        row = self.meta.iloc[idx]
        image_id = row['image_id']

        # 读取图像（灰度）
        img = Image.open(self.img_dir / f'{image_id}.png').convert('L')
        # 读取 mask（灰度）
        mask = Image.open(self.msk_dir / f'{image_id}.png').convert('L')

        img = self.transform(img)  # [1,H,W]
        mask = self.transform(mask)
        mask = (mask > 0.5).float()  # 二值化

        # 元数据向量：age/sex/modality -> float 张量
        # 把元数据拼成向量
        meta = torch.tensor([row['age'], row['sex'], row['modality']], dtype=torch.float32)
        # 简单归一化（实际项目建议使用训练集均值/方差）
        meta[0] = meta[0] / 100.0
        meta[2] = meta[2] / 2.0

        return img, mask, meta

DATASET_ROOT = 'examples/transformer_tutorial2/data/resnet_tutorial'
dataset = MultimodalSegDataset(DATASET_ROOT)
img, mask, meta = dataset[0]
img.shape, mask.shape, meta


## 3. 构建模型：ResNet 编码器 + 轻量解码器（带元数据注入）

思路：
1. 用 ResNet18 把图像编码为特征图。
2. 用元数据做一个小 MLP，得到与特征通道匹配的向量。
3. **把元数据向量加到特征图上**（即“多模态融合”）。
4. 用解码器上采样回到原图大小，输出分割 mask。


In [ ]:
import torch.nn as nn
import torchvision.models as models

class ResNetSeg(nn.Module):
    def __init__(self, meta_dim=3):
        super().__init__()
        backbone = models.resnet18(weights=None)
        self.encoder = nn.Sequential(*list(backbone.children())[:-2])  # [B,512,4,4] for 128x128

        self.meta_mlp = nn.Sequential(
            nn.Linear(meta_dim, 64), nn.ReLU(),
            nn.Linear(64, 512)
        )

        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(512, 256, 2, 2),  # 8x8
            nn.ReLU(),
            nn.ConvTranspose2d(256, 128, 2, 2),  # 16x16
            nn.ReLU(),
            nn.ConvTranspose2d(128, 64, 2, 2),   # 32x32
            nn.ReLU(),
            nn.ConvTranspose2d(64, 32, 2, 2),    # 64x64
            nn.ReLU(),
            nn.ConvTranspose2d(32, 16, 2, 2),    # 128x128
            nn.ReLU(),
            nn.Conv2d(16, 1, kernel_size=1)
        )

    def forward(self, x, meta):
        feat = self.encoder(x)
        meta_embed = self.meta_mlp(meta).unsqueeze(-1).unsqueeze(-1)
        feat = feat + meta_embed  # 多模态融合
        logits = self.decoder(feat)
        return logits

model = ResNetSeg()
model(img.unsqueeze(0), meta.unsqueeze(0)).shape


## 3.1 快速运行自检（确保模型能前向）

这一小段不会训练，只做**快速检查**：
- 模型能前向
- 输出尺寸正确


In [ ]:
# ✅ 快速自检：模型前向 + 输出尺寸
img, mask, meta = dataset[0]
model = ResNetSeg()
logits = model(img.unsqueeze(0), meta.unsqueeze(0))
print('logits shape:', logits.shape)
assert logits.shape[-2:] == mask.shape[-2:], '输出尺寸应与 mask 一致'


## 4. 训练准备

我们使用：
- 损失函数：`BCE + Dice`（分割常用）
- 优化器：Adam
- 指标：Dice 系数、IoU


In [ ]:
from torch.optim import Adam

def dice_loss(logits, targets, eps=1e-6):
    probs = torch.sigmoid(logits)
    num = 2 * (probs * targets).sum(dim=(1,2,3))
    den = probs.sum(dim=(1,2,3)) + targets.sum(dim=(1,2,3)) + eps
    return 1 - (num / den).mean()

def iou_score(logits, targets, eps=1e-6):
    probs = (torch.sigmoid(logits) > 0.5).float()
    inter = (probs * targets).sum(dim=(1,2,3))
    union = (probs + targets - probs*targets).sum(dim=(1,2,3))
    return ((inter + eps) / (union + eps)).mean().item()

loader = DataLoader(dataset, batch_size=4, shuffle=True)
model = ResNetSeg()
optimizer = Adam(model.parameters(), lr=1e-3)


## 5. 训练循环（最小可运行版）

> 这里只跑 2 个 epoch，确保你能“快速跑通”。


In [ ]:
from tqdm import tqdm

model.train()
for epoch in range(2):
    epoch_loss = 0
    for imgs, masks, metas in tqdm(loader, desc=f'Epoch {epoch+1}'):
        # 前向传播
        logits = model(imgs, metas)
        # 计算损失（BCE + Dice）
        loss = nn.functional.binary_cross_entropy_with_logits(logits, masks) + dice_loss(logits, masks)

        # 反向传播与更新参数
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
    print('avg loss:', epoch_loss / len(loader))


## 6. 评估与可视化

我们看看单张图像的预测效果（可视化）。


In [ ]:
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    img, mask, meta = dataset[0]
    pred = torch.sigmoid(model(img.unsqueeze(0), meta.unsqueeze(0)))[0,0]

    plt.figure(figsize=(8,3))
    plt.subplot(1,3,1); plt.title('Image'); plt.imshow(img[0], cmap='gray')
    plt.subplot(1,3,2); plt.title('GT Mask'); plt.imshow(mask[0], cmap='gray')
    plt.subplot(1,3,3); plt.title('Pred'); plt.imshow(pred, cmap='gray')
    plt.show()

    print('IoU:', iou_score(pred.unsqueeze(0).unsqueeze(0), mask.unsqueeze(0)))


## 7. 用你自己的数据怎么做？

### 7.1 数据摆放
把你的数据替换到：
```
examples/transformer_tutorial2/data/resnet_tutorial/
  images/
  masks/
  metadata.csv
```

### 7.2 需要注意的点
1. **图像和 mask 必须一一对应**。
2. `metadata.csv` 的 `image_id` 要和文件名一致（不带扩展名）。
3. 如果你有多个模态（CT/MRI/US），可以把它当成一个数值类别（0/1/2）。

### 7.3 预训练与微调
- **预训练**：可以先在公开数据集（如 ACDC、BraTS）上训练。
- **微调**：把学习率调小（如 1e-4），训练轮数调少，继续在你的私有数据上训练。

### 7.4 进一步提升
- 替换更强的解码器（如 U-Net / FPN / DeepLab）。
- 加入更丰富的元数据（文本报告、临床指标）。
